## Cadenas en LangChain

Las cadenas en LangChain permiten crear flujos de trabajo, donde se conectan distintos "bloques" para construir sistemas con modelos de lenguaje más complejos.

Por ejemplo, si necesitas un sistema que conecte múltiples entradas y salidas entre LLMs, las cadenas te permiten gestionar qué modelo genera qué información, con qué prompt, y cómo la salida de un modelo puede usarse como entrada de otro.
Cadenas más usadas

LangChain incluye diversas cadenas integradas, pero estas son las más comunes para desarrollar sistemas complejos:

1. LLMChain
2. SequentialChain
3. Math/Transformation

### 1. LLMChain

La LLMChain es una de las cadenas más utilizadas. Facilita la interacción con los modelos al unir dos elementos clave:

- Un modelo LLM (como OpenAI, LLama, Cohere, etc.)
. Un template de prompt.


En este ejemplo, el modelo actúa como un asistente para estudiantes y devuelve una lista de tres puntos clave sobre un tema académico.

In [ ]:
from langchain import LLMChain, OpenAI, PromptTemplate
import config  # Archivo con la API key

# Template modificado para un asistente académico
prompt = '''Eres un asistente para estudiantes que explica {tema} 
            enumerando 3 puntos clave de forma clara y precisa.
            Solo menciona los tres puntos.'''
template = PromptTemplate.from_template(prompt)

# Configuración del modelo usando API key desde config.py
llm = OpenAI(openai_api_key=config.OPENAI_API_KEY)

# Creación de la cadena LLM
cadena_LLM = LLMChain(llm=llm, prompt=template)

# Predicción con la cadena
output = cadena_LLM.predict(tema="la fotosíntesis") #Se le pasa mediante el .predict (veremos en secuencia de cadenas que se pasa como diccionario)
print(output)


### 2. SequentialChain en LangChain

Para muchos casos de uso, enviar un texto para procesarlo no es suficiente. A menudo, se requiere una secuencia de procesos que se ejecuten en orden, donde la salida de un modelo funcione como entrada para otro. LangChain ofrece dos opciones principales para esto:

- SimpleSequentialChain: Más básica, maneja un solo flujo lineal de entrada/salida.
- SequentialChain: Más flexible, permite manejar múltiples entradas y salidas.

En este ejemplo, usaremos SequentialChain, ya que ofrece mayor flexibilidad y la capacidad de recibir múltiples variables.

In [ ]:
from langchain import LLMChain, PromptTemplate
import config  # API key desde un archivo config.py
from langchain.chains import SequentialChain

# Prompt para generar una lista de conceptos clave
prompt_lista = '''Eres un asistente para estudiantes que explica {tema} 
                  enumerando 3 puntos clave de forma clara y precisa.
                  Solo menciona los tres puntos.'''
template_lista = PromptTemplate.from_template(prompt_lista)

# Creación de la primera cadena
llm = OpenAI(openai_api_key=config.API_KEY)
cadena_lista = LLMChain(llm=llm, prompt=template_lista, output_key="lista_conceptos")


In [ ]:
# Prompt para decidir por dónde iniciar el aprendizaje
prompt_inicio = '''Eres un asistente académico que analiza una lista de conceptos 
                   clave y decide cuál es mejor aprender primero.
                   Los conceptos son: {lista_conceptos}. Indica solo uno.'''
template_inicio = PromptTemplate.from_template(prompt_inicio)

# Creación de la segunda cadena
cadena_inicio = LLMChain(llm=llm, prompt=template_inicio, output_key="donde_iniciar")


In [ ]:
# Encadenamiento de procesos
cadenas = SequentialChain(
    chains=[cadena_lista, cadena_inicio],
    input_variables=["tema"],  # Variables de entrada iniciales
    output_variables=["lista_conceptos", "donde_iniciar"],  # Variables de salida finales
    verbose=True
)

# Ejecución con un tema de prueba
resultado = cadenas({"tema": "la fotosíntesis"}) #se llama directamente a la función y no se usa .predict, se le pasa un diccionario
print(resultado)


SimpleSequentialChain

Para un flujo más básico, se puede usar SimpleSequentialChain, aunque no permite múltiples entradas ni salidas.

In [ ]:
from langchain.chains import SimpleSequentialChain

# Encadenamiento simple
cadena_simple = SimpleSequentialChain(
    chains=[cadena_lista, cadena_inicio], 
    verbose=True
)

# Ejecución con un tema diferente
resultado_simple = cadena_simple.run("Inteligencia Artificial")
print(resultado_simple)


### 3. Otros Ejemplos de Cadenas

LangChain ofrece herramientas específicas como MathChain y TransformChain para cubrir necesidades comunes. Vamos a explorar estos ejemplos:

#### MathChain

- Diseñada para realizar cálculos matemáticos.
- Integra LLMs para interpretar y resolver problemas matemáticos.
- Utiliza herramientas como Python o entornos simbólicos para garantizar la precisión en los cálculos.

In [ ]:
from langchain import LLMMathChain

# Crear la cadena para cálculos matemáticos
cadena_mate = LLMMathChain(llm=llm, verbose=True)

# Realizar un cálculo
resultado = cadena_mate.run("Cuánto es 432 * 12 - 32 + 32?")
print(resultado)


#### TransformChain

- Permite realizar transformaciones personalizadas en los datos.
- Ideal para manipular texto, convertir formatos o realizar preprocesamientos personalizados.
- Utilizas una función Python personalizada para definir la transformación.

In [ ]:
from langchain.chains import TransformChain

# Definir una función para eliminar saltos de línea
def eliminar_brincos(input):
    """Elimina los brincos de línea de un texto."""
    texto = input["texto"]
    return {"texto_limpio": texto.replace("\n", " ")}

# Crear la cadena de transformación
cadena_transformacion = TransformChain(
    input_variables=["texto"],
    output_variables=["texto_limpio"],
    transform=eliminar_brincos
)

# Definir un texto con saltos de línea
prompt = '''\nEste es un texto \ncon brincos de\n línea.\n\n'''

# Ejecutar la transformación
resultado = cadena_transformacion.run({"texto": prompt})
print(resultado)
